In [2]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import time

# Mediapipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,         # 실시간 추적 모드
    max_num_hands=2,                 # 최대 두 손 추적
    min_detection_confidence=0.6,    # 검출 최소 신뢰도
    min_tracking_confidence=0.6      # 추적 최소 신뢰도
)

# 제스처 이름 설정 (수집하려는 제스처에 맞게 변경)
gesture_name = "추워요"

# 데이터 저장 경로 설정
output_dir = "hand_data"
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 생성

# 수집 횟수 초기화
file_count = 1
max_files = 100  # 총 100개의 파일을 저장

# 제스처 스팟팅 관련 변수 초기화
is_collecting = False
start_time = None

# 손 검출 상태 추적 변수
no_hand_frame_count = 0
hand_frame_count = 0
hand_presence_threshold = 5  # 손이 감지되거나 사라졌다고 판단하는 프레임 수

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened() and file_count <= max_files:
    success, image = cap.read()
    if not success:
        print("카메라 입력을 읽을 수 없습니다.")
        break

    # Mediapipe 처리
    image = cv2.flip(image, 1)  # 좌우 반전
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    # 손이 감지되었는지 확인
    hand_detected = results.multi_hand_landmarks is not None

    # 손 검출 상태 업데이트
    if hand_detected:
        hand_frame_count += 1
        no_hand_frame_count = 0
    else:
        no_hand_frame_count += 1
        hand_frame_count = 0

    # 제스처 스팟팅 로직 수정
    if hand_frame_count >= hand_presence_threshold and not is_collecting:
        # 손이 일정 프레임 이상 감지됨 - 데이터 수집 시작
        is_collecting = True
        start_time = time.time()
        # 저장할 CSV 파일 경로 설정
        file_path = os.path.join(output_dir, f"{gesture_name}_{file_count}.csv")  # 제스처 이름 포함
        # CSV 파일 생성 및 헤더 작성
        with open(file_path, mode="w", newline="") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["Time", "Hand", "ID", "X", "Y", "Z", "Gesture"])  # 'Gesture' 컬럼 추가
        print(f"{file_count}번 데이터 수집 시작 ({gesture_name})")
    elif no_hand_frame_count >= hand_presence_threshold and is_collecting:
        # 손이 일정 프레임 이상 감지되지 않음 - 데이터 수집 종료
        is_collecting = False
        print(f"{file_count}번 데이터 수집 종료 ({gesture_name})")
        file_count += 1  # 파일 수 증가

    # 데이터 수집 및 저장
    if is_collecting and hand_detected:
        # 현재 시간 기준으로 시간축 계산
        current_time = time.time() - start_time

        # 각 손의 랜드마크 데이터 저장
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # 손의 이름 (왼손/오른손)
            handedness = results.multi_handedness[idx].classification[0].label
            handedness = "Left" if handedness == "Left" else "Right"

            # 저장할 CSV 파일 경로
            file_path = os.path.join(output_dir, f"{gesture_name}_{file_count}.csv")  # 제스처 이름 포함
            with open(file_path, mode="a", newline="") as csv_file:
                writer = csv.writer(csv_file)
                for landmark_idx, landmark in enumerate(hand_landmarks.landmark):
                    writer.writerow([
                        current_time,     # 시간축
                        handedness,       # 왼손/오른손
                        landmark_idx,     # 랜드마크 ID
                        landmark.x,       # X 좌표
                        landmark.y,       # Y 좌표
                        landmark.z,       # Z 좌표
                        gesture_name      # 제스처 이름 (라벨) 추가
                    ])

    # 랜드마크 그리기
    if hand_detected:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # 화면 표시
    cv2.putText(image, f"Collecting: {is_collecting}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, f"File Count: {file_count}/{max_files}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Hand Tracking", image)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1번 데이터 수집 시작 (추워요)
1번 데이터 수집 종료 (추워요)
2번 데이터 수집 시작 (추워요)
2번 데이터 수집 종료 (추워요)
3번 데이터 수집 시작 (추워요)
3번 데이터 수집 종료 (추워요)
4번 데이터 수집 시작 (추워요)
4번 데이터 수집 종료 (추워요)
5번 데이터 수집 시작 (추워요)
5번 데이터 수집 종료 (추워요)
6번 데이터 수집 시작 (추워요)
6번 데이터 수집 종료 (추워요)
7번 데이터 수집 시작 (추워요)
7번 데이터 수집 종료 (추워요)
8번 데이터 수집 시작 (추워요)
8번 데이터 수집 종료 (추워요)
9번 데이터 수집 시작 (추워요)
9번 데이터 수집 종료 (추워요)
10번 데이터 수집 시작 (추워요)
10번 데이터 수집 종료 (추워요)
11번 데이터 수집 시작 (추워요)
11번 데이터 수집 종료 (추워요)
12번 데이터 수집 시작 (추워요)
12번 데이터 수집 종료 (추워요)
13번 데이터 수집 시작 (추워요)
13번 데이터 수집 종료 (추워요)
14번 데이터 수집 시작 (추워요)
14번 데이터 수집 종료 (추워요)
15번 데이터 수집 시작 (추워요)
15번 데이터 수집 종료 (추워요)
16번 데이터 수집 시작 (추워요)
16번 데이터 수집 종료 (추워요)
17번 데이터 수집 시작 (추워요)
17번 데이터 수집 종료 (추워요)
18번 데이터 수집 시작 (추워요)
18번 데이터 수집 종료 (추워요)
19번 데이터 수집 시작 (추워요)
19번 데이터 수집 종료 (추워요)
20번 데이터 수집 시작 (추워요)
20번 데이터 수집 종료 (추워요)
21번 데이터 수집 시작 (추워요)
21번 데이터 수집 종료 (추워요)
22번 데이터 수집 시작 (추워요)
22번 데이터 수집 종료 (추워요)
23번 데이터 수집 시작 (추워요)
23번 데이터 수집 종료 (추워요)
24번 데이터 수집 시작 (추워요)
24번 데이터 수집 종료 (추워요)
25번 데이터 수집 시작 (추워요)
25번 데이터 수집 종료 (추워요)
26번 데이터 수집 시작 (추워요